In [20]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
from sklearn.model_selection import train_test_split
import cv2
import sklearn
import random
import os.path as ops

# Build the data Generator with data augmentation

In [46]:
def generator(samples, batch_size=32, data_folder_path=None):
    num_samples = len(samples)
    while 1:  # Loop forever so the generator never terminates
        random.shuffle(samples)
        for offset in range(0, num_samples, int(batch_size / 2)):
            batch_samples = samples[offset:offset + int(batch_size / 2)]
            images = []
            angles = []
            for batch_sample in batch_samples:
                name = ops.join(data_folder_path, batch_sample[0])
                center_image = cv2.imread(name)
                center_angle = float(batch_sample[3])

                # flip the center image and angle to do data augmentation
                center_image_flip = cv2.flip(center_image, 1)
                center_angle_flip = -center_angle

                center_image = cv2.cvtColor(center_image, cv2.COLOR_BGR2RGB)
                center_image_flip = cv2.cvtColor(center_image_flip, cv2.COLOR_BGR2RGB)

                images.append(center_image)
                angles.append(center_angle)
                images.append(center_image_flip)
                angles.append(center_angle_flip)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

In [49]:
data_folder_path = '/Users/smiffy/Documents/GitHub/project3_data'
samples = []
for data_folder in os.listdir(data_folder_path):
    if data_folder == '.DS_Store':
        continue
    elif data_folder == 'data':
        with open(data_folder_path + '/' + data_folder + '/driving_log.csv') as csvfile:
            reader = csv.reader(csvfile)
            for i, line in enumerate(reader):
                if i != 0:
                    line[0] = ops.join(data_folder, 'IMG', ops.split(line[0])[-1])
                    line[1] = ops.join(data_folder, 'IMG', ops.split(line[1])[-1])
                    line[2] = ops.join(data_folder, 'IMG', ops.split(line[2])[-1])
                    samples.append(line)
                else:
                    pass
    else:
        with open(data_folder_path + '/' + data_folder + '/driving_log.csv') as csvfile:
            reader = csv.reader(csvfile)
            for line in reader:
                line[0] = ops.join(data_folder, 'IMG', ops.split(line[0])[-1])
                line[1] = ops.join(data_folder, 'IMG', ops.split(line[1])[-1])
                line[2] = ops.join(data_folder, 'IMG', ops.split(line[2])[-1])
                samples.append(line)

random.shuffle(samples)
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [50]:
print('samples length: {}'.format(len(samples)))
print('train_samples length: {}'.format(len(train_samples)))
print('validation_samples length: {}'.format(len(validation_samples)))

samples length: 11771
train_samples length: 9416
validation_samples length: 2355


In [52]:
# Set our batch size
batch_size = 32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size, data_folder_path=data_folder_path)
validation_generator = generator(validation_samples, batch_size=batch_size, data_folder_path=data_folder_path)

In [53]:
for batch in train_generator:
    break